In [17]:
import os
import pandas as pd
import numpy as np

carpeta_tweets = "./tweets"

categorias_posibles = ["Violencia", "Homofobia", "Xenofobia", "Incensurable"]

dataframes_por_archivo = {}

for nombre_archivo in os.listdir(carpeta_tweets):
    if nombre_archivo.endswith(".txt"):
        ruta_archivo = os.path.join(carpeta_tweets, nombre_archivo)

        datos = []
        with open(ruta_archivo, mode='r', encoding='utf-8') as archivo:
            for linea in archivo:
                partes = linea.strip().split("#############")
                if len(partes) != 3:
                    continue

                texto, etiquetas, tweet_padre = partes

                etiquetas_set = set(et.strip() for et in etiquetas.split(",") if et.strip())
                if not etiquetas_set:
                    etiquetas_set = {"Incensurable"}

                fila = {
                    "Tweet": texto,
                    "Tweet_Padre": tweet_padre
                }

                for cat in categorias_posibles:
                    fila[cat] = int(cat in etiquetas_set)

                datos.append(fila)

        nombre_df = os.path.splitext(nombre_archivo)[0]
        df = pd.DataFrame(datos)
        dataframes_por_archivo[nombre_df] = df

In [18]:
df_unificado = pd.concat(dataframes_por_archivo.values(), ignore_index=True)
df_unificado.head()

,Tweet,Tweet_Padre,Violencia,Homofobia,Xenofobia,Incensurable
0,No queremos más venecos en Chile,SinTweetPadre,0,0,1,0
1,El negro Piñera se murió hace dos meses y ya N...,SinTweetPadre,0,0,0,1
2,“De venir de la calle …a vivir de lujos gracia...,SinTweetPadre,0,0,0,1
3,"5:47 y ya está oscuro, ¿esa wea querian los in...",SinTweetPadre,0,0,0,1
4,Uno se pregunta como negros qls pobres q vagan...,SinTweetPadre,0,0,1,0


In [19]:
tweets_duplicados = df_unificado[df_unificado.duplicated(subset='Tweet', keep=False)]
grupos = tweets_duplicados.groupby('Tweet')
indices_a_eliminar = []

for tweet, grupo in grupos:
    if len(grupo) >= 22:
        columnas_comparar = [col for col in df_unificado.columns if col != 'ID']
        if grupo[columnas_comparar].nunique().sum() == len(columnas_comparar):
            indices_a_eliminar.extend(grupo.index[1:].tolist())
        else:
            print(f"❗ Tweets repetidos:\n{tweet}")
            print(grupo)
            print("\n---\n")

df_unificado = df_unificado.drop(index=indices_a_eliminar).reset_index(drop=True)

print(f"Se eliminaron {len(indices_a_eliminar)} duplicados idénticos.")



Se eliminaron 0 duplicados idénticos.


In [20]:
df_unificado['ID'] = range(1, len(df_unificado) + 1)

columnas_ordenadas = ['ID'] + [col for col in df_unificado.columns if col != 'ID']
df_unificado = df_unificado[columnas_ordenadas]

df_unificado.head()


,ID,Tweet,Tweet_Padre,Violencia,Homofobia,Xenofobia,Incensurable
0,1,No queremos más venecos en Chile,SinTweetPadre,0,0,1,0
1,2,El negro Piñera se murió hace dos meses y ya N...,SinTweetPadre,0,0,0,1
2,3,“De venir de la calle …a vivir de lujos gracia...,SinTweetPadre,0,0,0,1
3,4,"5:47 y ya está oscuro, ¿esa wea querian los in...",SinTweetPadre,0,0,0,1
4,5,Uno se pregunta como negros qls pobres q vagan...,SinTweetPadre,0,0,1,0


In [21]:
texto_a_id = dict(zip(df_unificado['Tweet'], df_unificado['ID']))

def obtener_id_padre(texto_padre):
    if texto_padre == "SinTweetPadre":
        return np.nan
    return texto_a_id.get(texto_padre, "X")

df_unificado['ID_Tweet_Padre'] = df_unificado['Tweet_Padre'].apply(obtener_id_padre)

num_casos_X = (df_unificado['ID_Tweet_Padre'] == "X").sum()
print(f"Número de tweets con 'X' en ID_Tweet_Padre (no se encontró el tweet padre): {num_casos_X}")

df_unificado.head()


Número de tweets con 'X' en ID_Tweet_Padre (no se encontró el tweet padre): 92


,ID,Tweet,Tweet_Padre,Violencia,Homofobia,Xenofobia,Incensurable,ID_Tweet_Padre
0,1,No queremos más venecos en Chile,SinTweetPadre,0,0,1,0,NaN
1,2,El negro Piñera se murió hace dos meses y ya N...,SinTweetPadre,0,0,0,1,NaN
2,3,“De venir de la calle …a vivir de lujos gracia...,SinTweetPadre,0,0,0,1,NaN
3,4,"5:47 y ya está oscuro, ¿esa wea querian los in...",SinTweetPadre,0,0,0,1,NaN
4,5,Uno se pregunta como negros qls pobres q vagan...,SinTweetPadre,0,0,1,0,NaN


In [22]:
import ipywidgets as widgets
from IPython.display import display, clear_output

tweets_padres_faltantes = df_unificado[df_unificado['ID_Tweet_Padre'] == "X"]['Tweet_Padre'].unique().tolist()

nuevos_tweets = []

categorias_posibles = ["Violencia", "Homofobia", "Xenofobia", "Incensurable"]

index = 0

texto_output = widgets.Output()
dropdown_categorias = widgets.SelectMultiple(
    options=categorias_posibles,
    description='Categorías:',
    style={'description_width': 'initial'}
)
boton_guardar = widgets.Button(description="Guardar y siguiente")
boton_omitir = widgets.Button(description="Omitir")
info_label = widgets.Label()

def mostrar_tweet():
    with texto_output:
        clear_output()
        if index < len(tweets_padres_faltantes):
            print(f"Tweet padre no encontrado ({index + 1}/{len(tweets_padres_faltantes)}):")
            print(tweets_padres_faltantes[index])
        else:
            print("¡Todos los tweets padres han sido procesados!")

def guardar_tweet(b):
    global index
    if index < len(tweets_padres_faltantes):
        texto = tweets_padres_faltantes[index]
        categorias = list(dropdown_categorias.value)
        etiquetas = categorias if categorias else ["Incensurable"]
        nuevo_id = df_unificado['ID'].max() + 1

        fila = {
            "ID": nuevo_id,
            "Tweet": texto,
            "Tweet_Padre": "SinTweetPadre",
            "ID_Tweet_Padre": np.nan
        }
        for cat in categorias_posibles:
            fila[cat] = int(cat in etiquetas)

        nuevos_tweets.append(fila)
        index += 1
        mostrar_tweet()

def omitir_tweet(b):
    global index
    index += 1
    mostrar_tweet()

boton_guardar.on_click(guardar_tweet)
boton_omitir.on_click(omitir_tweet)

mostrar_tweet()
display(texto_output, dropdown_categorias, boton_guardar, boton_omitir, info_label)


Output()

SelectMultiple(description='Categorías:', options=('Violencia', 'Homofobia', 'Xenofobia', 'Incensurable'), sty…

Button(description='Guardar y siguiente', style=ButtonStyle())

Button(description='Omitir', style=ButtonStyle())

Label(value='')

In [23]:
# Agregar los nuevos tweets al DataFrame
df_unificado = pd.concat([df_unificado, pd.DataFrame(nuevos_tweets)], ignore_index=True)

# Volver a crear el mapeo de texto -> ID
texto_a_id = dict(zip(df_unificado['Tweet'], df_unificado['ID']))

# Actualizar ID_Tweet_Padre nuevamente
df_unificado['ID_Tweet_Padre'] = df_unificado['Tweet_Padre'].apply(
    lambda x: np.nan if x == "SinTweetPadre" else texto_a_id.get(x, "X")
)

# Verificar si aún quedan tweets con padre no encontrado
print(f"Tweets con 'X' restantes: {(df_unificado['ID_Tweet_Padre'] == 'X').sum()}")


Tweets con 'X' restantes: 92
